In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
from scipy.signal import correlate2d
from sklearn.metrics import confusion_matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from bayesee.generation import *

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
subject = 'AZ_t1'
file_name = repo_path / f'data/covert-search/large-field/est_dp_{subject}.npy'
local_dp = np.load(file_name)
print(local_dp)

In [ ]:
file_name = repo_path / f'data/covert-search/large-field/p3_data_{subject}.mat'
data = loadmat(file_name)
print(data.keys())

In [ ]:
target_amplitude = data['targetAmplitude']
spot_center = data['spotCenters']
n_location = data['nLocations'][0][0]
spot_diameter = data['spotLength'][0][0]
stimulus_design_size= data['totalLength'][0][0]
monitor_width = data['monitorPx'][0][0]
monitor_height = data['monitorPx'][0][1]
ppd = data['ppd'][0][0]
target = data['targets']
background_mean = data['bgMean'][0][0]
background_std = data['bgContrast'][0][0] * background_mean

In [ ]:
shifted_spot_center = spot_center.copy()
shifted_spot_center[:,0] += (monitor_height - stimulus_design_size)//2
shifted_spot_center[:,1] += (monitor_width - stimulus_design_size)//2

In [ ]:
print(f'N Location:{n_location-1}\nTarget Size:{target.shape}\nBackground Mean:{background_mean}\nBackground STD:{background_std}')

In [ ]:
def infinite_white_noises(n=19, size=(30,30), mean=128, std=25.6):
    while True:
        noises = np.random.normal(size=(n, *size))
        yield std*noises/noises.std()+mean

In [ ]:
prior = np.array((0.5, *((0.5/(n_location-1),)*(n_location-1))))
assert np.allclose(prior.sum(), 1.0)
log_prior = np.log(prior)
log_likelihood = np.ones_like(prior)

In [ ]:
n_trials = 800
the_target_amplitude = target_amplitude[0,0,0]

target_location = np.zeros((n_trials,))
target_location[:n_trials//2] = np.random.randint(1,n_location,size=n_trials//2)
model_response = np.zeros_like(target_location)

assert np.allclose(np.dot(target.flatten(), target.flatten()),1.0)

for index_trial in range(n_trials):
    white_noises = next(infinite_white_noises())
    for index_location in range(n_location-1):
        effect_background_std = the_target_amplitude / local_dp[index_location]
        effect_background = white_noises[index_location,:,:].flatten() / background_std * effect_background_std

        log_likelihood[index_location+1] = the_target_amplitude / effect_background_std**2 * (np.dot(target.flatten(), effect_background)-the_target_amplitude/2)
        
        if target_location[index_trial] == index_location+1:
            log_likelihood[index_location+1] += local_dp[index_location]**2

    log_posterior = log_prior + log_likelihood
    model_response[index_trial] = np.argmax(log_posterior)

In [ ]:
list_spot_region = [np.zeros(monitor_width,monitor_height)] * (n_location-1)
pixel_col, pixel_row = np.meshgrid(np.arange(monitor_width), np.arange(monitor_height))

for index_location in range(n_location-1):
     list_spot_region[index_location] = (pixel_row - shifted_spot_center[index_location, 0])**2+(pixel_col - shifted_spot_center[index_location, 1])**2<= spot_diameter**2/4

In [ ]:
stimulus_region = np.zeros((monitor_height, monitor_width))

for index_location in range(n_location-1):
    stimulus_region[list_spot_region[index_location]] = 1

In [ ]:
array_eccentral_distance = np.zeros((n_location-1,))

for index_location in range(n_location-1):
    array_eccentral_distance[index_location] = np.sqrt((spot_center[index_location, 0] - stimulus_design_size//2)**2+(spot_center[index_location, 1] - stimulus_design_size//2)**2)

array_eccentral_distance /= ppd

print(array_eccentral_distance)

In [ ]:
pixel_precision_array_eccentral_distance = array_eccentral_distance.copy()

for index_d1, distance1 in enumerate(np.unique(array_eccentral_distance)):
    for index_d2, distance2 in enumerate(np.unique(array_eccentral_distance)[index_d1+1:]):
        if distance1 != distance2 and np.abs(distance2 - distance1) < 0.5:
            print(index_d1, distance1, index_d2, distance2)
            pixel_precision_array_eccentral_distance[array_eccentral_distance==distance2] = distance1

print(pixel_precision_array_eccentral_distance)

In [ ]:
eccentral_distance_sorted_indexes = np.argsort(pixel_precision_array_eccentral_distance, kind='stable')
extra_eccentral_distance_sorted_indexes = np.insert(eccentral_distance_sorted_indexes+1,0,0)
print(extra_eccentral_distance_sorted_indexes)

In [ ]:
accurate_response = target_location == model_response
confusion_mat = confusion_matrix(model_response.flatten(), target_location.flatten(), labels=range(n_location))[extra_eccentral_distance_sorted_indexes, :][:, extra_eccentral_distance_sorted_indexes]
array_index_location = np.arange(n_location)[extra_eccentral_distance_sorted_indexes]

In [ ]:
accurate_response = target_location == model_response
confusion_mat = confusion_matrix(model_response.flatten(), target_location.flatten(), labels=range(n_location))[extra_eccentral_distance_sorted_indexes, :][:, extra_eccentral_distance_sorted_indexes]
array_index_location = np.arange(n_location)[extra_eccentral_distance_sorted_indexes]

In [ ]:
correct_rejection = confusion_mat[0][0]
second_largest_confusion_mat = np.max(np.delete(confusion_mat, np.where(confusion_mat == correct_rejection)))
plotted_confusion_mat = confusion_mat.copy()
plotted_confusion_mat[0][0] = second_largest_confusion_mat
print(second_largest_confusion_mat)

In [ ]:
fig, ax = plt.subplots(figsize=(15,12))
im = ax.imshow(plotted_confusion_mat)

for index_target_location in array_index_location:
    for index_model_response in array_index_location:
        if not (-index_target_location == 0 and index_model_response == 0):
            ax.text(index_model_response, index_target_location, f'{confusion_mat[index_target_location, index_model_response]:.0f}', ha='center', va='center')

ax.annotate(f'{correct_rejection:.0f}', xy=(0, 0), xytext=(-0.5,-1), arrowprops=dict(facecolor='black', shrink=0.05))

ax.set(xticks=np.arange(n_location), yticks=np.arange(n_location), xticklabels=array_index_location, yticklabels=array_index_location, xlabel='Target location sorted by eccentral distance', ylabel='Model response sorted by eccentral distance')

plt.show()